In [42]:
from functools import partial
from sqlite3 import connect

# import astroplan as ap
# from astropy.coordinates import EarthLocation
# from astropy.table import Table
# from astropy.time import Time
# import astropy.units as u
# from icecream import ic, install
# import numpy as np
import pandas as pd

import astropaul.targetlistcreator as tlc
import astropaul.lbt as lbt
import astropaul.html as html
import astropaul.phase as ph
# import astropaul.priority as pr

# install() # icecream will now work in imported packages

%load_ext autoreload
%autoreload 2


The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [46]:
conn = connect("../../Data/astropaul.db")

phase_event_defs = [
    ph.PhaseEventDef("Not in Eclipse", partial(ph.calc_time_of_gress, ingress=False)),
    ph.PhaseEventDef("Eclipse", partial(ph.calc_time_of_gress, ingress=True)),
]

name = "All Targets"
creator = tlc.TargetListCreator(name=name, connection=conn, phase_event_defs=phase_event_defs)
creator.steps = [
    tlc.add_targets,
    tlc.ancillary_data_from_tess,
    lbt.add_rv_calibration_targets,
    tlc.add_catalogs,
    tlc.add_lists,
    partial(tlc.add_database_table, table_name="ephemerides"),
    partial(tlc.add_database_table, table_name="dssi_observations"),
    partial(tlc.add_database_table, table_name="speckle_detections"),
    partial(tlc.add_database_table, table_name="pepsi_observations"),
    partial(
        tlc.add_system_configuration, table_name="DSSI Observations", time_column="Mid JD", eclipse_table="SIDE Observations"
    ),
    partial(tlc.add_system_configuration, table_name="PEPSI Observations", time_column="Mid JD"),
]
tl = creator.calculate(verbose=False)
html.render_observing_pages(tl, None, {}, "../../Observing Files/All Targets")
conn.close()